In [10]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
Pkg.add("XLSX")


  Activating project at `~/Desktop/BEE 4750/BEE4750FinalProject-1`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed ZipFile ──────── v0.10.1
   Installed XML2_jll ─────── v2.15.1+0
   Installed InputBuffers ─── v1.1.1
   Installed CodecInflate64 ─ v0.1.3
   Installed EzXML ────────── v1.2.3
   Installed ZipArchives ──── v2.6.0
   Installed XLSX ─────────── v0.10.4
    Updating `~/Desktop/BEE 4750/BEE4750FinalProject-1/Project.toml`
  [fdbf4ff8] + XLSX v0.10.4
    Updating `~/Desktop/BEE 4750/BEE4750FinalProject-1/Manifest.toml`
  [6309b1aa] + CodecInflate64 v0.1.3
  [8f5d6c58] + EzXML v1.2.3
  [0c81fc1b] + InputBuffers v1.1.1
  [fdbf4ff8] + XLSX v0.10.4
  [49080126] + ZipArchives v2.6.0
  [a5390f91] + ZipFile v0.10.1
  [02c8fc9c] + XML2_jll v2.15.1+0
Precompiling project...
   1683.2 ms  ✓ InputBuffers
   1021.8 ms  ✓ ZipFile
    977.4 ms  ✓ XML2_jll
   1036.2 ms  ✓ CodecInflate64
   1201.8 ms  ✓ EzXML
   3700.0 ms  ✓ ZipArchives

In [11]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables
using XLSX

In [12]:
filepath = "PSB.xlsx"
sheetname = "Exported Data" # Or the name of your desired sheet
df = DataFrame(XLSX.readtable(filepath, sheetname))
column_name = :Date # Replace with the actual column name (as a Symbol)
desired_column = df[:, column_name] # Access the column as a Vector


ArgumentError: ArgumentError: column name "Date" not found in the data frame; existing most similar names are: "name"

In [19]:
# Read the table directly into a DataTable object first
datatable = XLSX.readtable("PSB.xlsx", "Exported Data")

# Convert the DataTable to a DataFrame directly (no splatting ...)
df = DataFrame(datatable)

# Now you can access column G by its header name
# Replace "Your_Column_G_Header" with the actual header text from cell G1
column_G_data = df."Date"

ArgumentError: ArgumentError: column name "Date" not found in the data frame; existing most similar names are: "name"